### The random forest in btc_aggregate outperforms this one

In [1]:
%who

Interactive namespace is empty.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings("ignore")

/Users/hanlinzhuang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv('../raw data/new/btc_hourly_ma.csv')

In [4]:
data['vol_per_count'] = data['volume_traded']/data['trades_count']

In [5]:
data['roi'] = np.log(data['price_close']/data['price_close'].shift(1))
data['roi_lag1'] = data['roi'].shift(1)

In [6]:
data['price_close_lag1'] = data['price_close'].shift(1)

In [7]:
data['volume_traded_lag1'] = data['volume_traded'].shift(1)
data['trades_count_lag1'] = data['trades_count'].shift(1)
data['vol_per_count_lag1'] = data['vol_per_count'].shift(1)

In [8]:
data['Label MA_lag1'] = data['Label MA'].shift(1)
data['Label MA_lag2'] = data['Label MA'].shift(2)
data['Label MA_lag3'] = data['Label MA'].shift(3)

In [9]:
label = ['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']
std = data['roi'].std()
mean = np.mean(data['roi'])
data['cut'] = pd.cut(data['roi'], 
                     bins=[-100, -3*std, -2*std, -std, mean, std, 2*std, 3*std, 100], 
                    labels=label)
data['cut_lag1'] = data['cut'].shift(1)
data['cut_lag2'] = data['cut'].shift(2)
data['cut_lag3'] = data['cut'].shift(3)

In [10]:
data.columns

Index(['Unnamed: 0', 'time_period_start', 'time_period_end', 'time_open',
       'time_close', 'price_open', 'price_high', 'price_low', 'price_close',
       'volume_traded', 'trades_count', 'Interval', 'Label MA',
       'vol_per_count', 'roi', 'roi_lag1', 'price_close_lag1',
       'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1',
       'Label MA_lag1', 'Label MA_lag2', 'Label MA_lag3', 'cut', 'cut_lag1',
       'cut_lag2', 'cut_lag3'],
      dtype='object')

In [11]:
data_ = data.dropna()
data_.head()

,Unnamed: 0,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,Label MA_lag1,Label MA_lag2,Label MA_lag3,cut,cut_lag1,cut_lag2,cut_lag3
147,147,2019-04-07 03:00:00.000,2019-04-07 04:00:00.000,2019-04-07 03:00:00.000,2019-04-07 03:59:59.984,5180.80,5748.51,5055.740696,5127.250000,4051.892976,...,3509.715085,7052.0,0.497691,3.750000,3.500000,4.000,-1,+2,-1,+1
148,148,2019-04-07 04:00:00.000,2019-04-07 05:00:00.000,2019-04-07 04:00:00.000,2019-04-07 04:59:59.979,5191.40,5742.23,5070.435963,5140.010000,2599.910887,...,4051.892976,8414.0,0.481566,3.400000,3.750000,3.500,+1,-1,+2,-1
149,149,2019-04-07 05:00:00.000,2019-04-07 06:00:00.000,2019-04-07 05:00:00.000,2019-04-07 05:59:59.000,5132.38,5726.97,4979.000000,5159.333672,2242.920435,...,2599.910887,6791.0,0.382847,3.375000,3.400000,3.750,+1,+1,-1,+2
150,150,2019-04-07 06:00:00.000,2019-04-07 07:00:00.000,2019-04-07 06:00:01.000,2019-04-07 06:59:59.000,5097.71,5714.68,5062.200010,5206.900000,1960.501063,...,2242.920435,7327.0,0.306117,3.454545,3.375000,3.400,+1,+1,+1,-1
151,151,2019-04-07 07:00:00.000,2019-04-07 08:00:00.000,2019-04-07 07:00:00.495,2019-04-07 07:59:59.000,5131.24,5736.16,5068.740000,5365.645307,2379.030114,...,1960.501063,5733.0,0.341968,3.416667,3.454545,3.375,+2,+1,+1,+1


In [12]:
data_['cut'].groupby(data_['cut']).count()

cut
-4     13
-3     11
-2     16
-1    243
+1    246
+2     22
+3     10
+4     12
Name: cut, dtype: int64

In [13]:
data_.shape

(573, 27)

In [14]:
y = data_['cut']
X = data_[['cut_lag1', 'cut_lag2', 'cut_lag3',
           'Label MA_lag1', 'Label MA_lag1', 'Label MA_lag1',
           'price_close_lag1', 
           'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
for i in X.columns.tolist()[-4:]:
    X[i] = StandardScaler().fit_transform(X[i].values.reshape(-1, 1))

In [17]:
X.head()

,cut_lag1,cut_lag2,cut_lag3,Label MA_lag1,Label MA_lag1,Label MA_lag1,price_close_lag1,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1
147,+2,-1,+1,3.750000,3.750000,3.750000,-0.464042,-0.145598,-0.140615,-0.083190
148,-1,+2,-1,3.400000,3.400000,3.400000,-0.789125,0.061142,0.140226,-0.247015
149,+1,-1,+2,3.375000,3.375000,3.375000,-0.711373,-0.492519,-0.194433,-1.249960
150,+1,+1,-1,3.454545,3.454545,3.454545,-0.593626,-0.628644,-0.083911,-2.029499
151,+1,+1,+1,3.416667,3.416667,3.416667,-0.303785,-0.736334,-0.412589,-1.665272


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [19]:
# random forest

In [20]:
clf_rf = RandomForestClassifier(n_estimators=1000, max_depth=4, bootstrap=True, random_state=0)
clf_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [21]:
print('accuracy:')
print(clf_rf.score(X_test, y_test))

accuracy:
0.5565217391304348


In [22]:
y_pred = clf_rf.predict(X_test)
y_pred

array(['+1', '-1', '+1', '-1', '-1', '-1', '+1', '+1', '+1', '+1', '-1',
       '+1', '+1', '+1', '-2', '-1', '+1', '+1', '-1', '-2', '+1', '+1',
       '+1', '+1', '-1', '+1', '-1', '-1', '-1', '-1', '+1', '-1', '-1',
       '+1', '+1', '+1', '+1', '+1', '+1', '+1', '-1', '+1', '-1', '-1',
       '+1', '-1', '-1', '+1', '-1', '+1', '+1', '-1', '-1', '+1', '-1',
       '+1', '-1', '+1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '+1',
       '-1', '-1', '+1', '-1', '-1', '+1', '-1', '-4', '+1', '+1', '-1',
       '-1', '-1', '+1', '-2', '+1', '-1', '-1', '-1', '-1', '+1', '+1',
       '+1', '-1', '+1', '+1', '-1', '+1', '-1', '-1', '+1', '+1', '+1',
       '-1', '-1', '-1', '+1', '+1', '-2', '-2', '+1', '-1', '-1', '-1',
       '+1', '-1', '+1', '-1', '+1'], dtype=object)

In [23]:
cmatrix = confusion_matrix(y_test, y_pred, labels=label)

In [24]:
confusionmatrix = pd.DataFrame(cmatrix, columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,1,0,0,0,0,0,0,0
Actual -3,0,0,1,0,0,0,0,0
Actual -2,0,0,3,1,0,0,0,0
Actual -1,0,0,1,30,20,0,0,0
Actual +1,0,0,0,18,30,0,0,0
Actual +2,0,0,0,4,1,0,0,0
Actual +3,0,0,0,0,3,0,0,0
Actual +4,0,0,0,2,0,0,0,0


In [25]:
print("Average precision for the 8 intervals is - ", precision_score(y_test, y_pred, labels=label, average = None))

Average precision for the 8 intervals is -  [1.         0.         0.6        0.54545455 0.55555556 0.
 0.         0.        ]


In [26]:
X.head()

,cut_lag1,cut_lag2,cut_lag3,Label MA_lag1,Label MA_lag1,Label MA_lag1,price_close_lag1,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1
147,+2,-1,+1,3.750000,3.750000,3.750000,-0.464042,-0.145598,-0.140615,-0.083190
148,-1,+2,-1,3.400000,3.400000,3.400000,-0.789125,0.061142,0.140226,-0.247015
149,+1,-1,+2,3.375000,3.375000,3.375000,-0.711373,-0.492519,-0.194433,-1.249960
150,+1,+1,-1,3.454545,3.454545,3.454545,-0.593626,-0.628644,-0.083911,-2.029499
151,+1,+1,+1,3.416667,3.416667,3.416667,-0.303785,-0.736334,-0.412589,-1.665272


In [27]:
data_[['cut', 'cut_lag1', 'cut_lag2', 'cut_lag3',
           'Label MA_lag1', 'Label MA_lag1', 'Label MA_lag1',
           'price_close_lag1', 
           'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']].head()

,cut,cut_lag1,cut_lag2,cut_lag3,Label MA_lag1,Label MA_lag1,Label MA_lag1,price_close_lag1,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1
147,-1,+2,-1,+1,3.750000,3.750000,3.750000,5180.600000,3509.715085,7052.0,0.497691
148,+1,-1,+2,-1,3.400000,3.400000,3.400000,5127.250000,4051.892976,8414.0,0.481566
149,+1,+1,-1,+2,3.375000,3.375000,3.375000,5140.010000,2599.910887,6791.0,0.382847
150,+1,+1,+1,-1,3.454545,3.454545,3.454545,5159.333672,2242.920435,7327.0,0.306117
151,+2,+1,+1,+1,3.416667,3.416667,3.416667,5206.900000,1960.501063,5733.0,0.341968
